In [74]:
text = ""
with open("./dataset/tinyshakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [75]:
print(f"Length of dataset: {len(text)}")

Length of dataset: 1115389


In [76]:
# Create a set of all the unique characters from the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars) } # This is really sexy
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # encoder: takes a string and return a list of integers
decode = lambda l: "".join([itos[ch] for ch in l]) # decoder: takes a list of int and return text

After this we make our tokenizer

In [77]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [78]:
# Split dataset
n = int(0.9*len(data)) # first 90% will be for training
train_data = data[:n]
test_data = data[n:]

block_size = 8

In [79]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1] # i+11 because we don't want 0 and range is exclusive
    target = y[i]
    # print(f"When input is {context} the target: {target}")

In [80]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # What is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else test_data
    random_offset = torch.randint(len(train_data) - block_size, (batch_size, ))
    contexts = torch.stack([data[i:i+block_size] for i in random_offset])
    targets = torch.stack([data[i+1:i+block_size+1] for i in random_offset])
    return contexts, targets

xb, yb = get_batch("train")
print("Inputs:")
print(xb.shape)
print(xb)

print("Outputs:")
print(yb.shape)
print(yb)

Inputs:
torch.Size([4, 8])
tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
Outputs:
torch.Size([4, 8])
tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])


In [128]:
from torch import nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) Tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            print(logits.shape)
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
            
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
# print(logits.shape)
# print(loss)

print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100).numpy()[0]))

torch.Size([1, 1, 65])
torch.Size([1, 2, 65])
torch.Size([1, 3, 65])
torch.Size([1, 4, 65])
torch.Size([1, 5, 65])
torch.Size([1, 6, 65])
torch.Size([1, 7, 65])
torch.Size([1, 8, 65])
torch.Size([1, 9, 65])
torch.Size([1, 10, 65])
torch.Size([1, 11, 65])
torch.Size([1, 12, 65])
torch.Size([1, 13, 65])
torch.Size([1, 14, 65])
torch.Size([1, 15, 65])
torch.Size([1, 16, 65])
torch.Size([1, 17, 65])
torch.Size([1, 18, 65])
torch.Size([1, 19, 65])
torch.Size([1, 20, 65])
torch.Size([1, 21, 65])
torch.Size([1, 22, 65])
torch.Size([1, 23, 65])
torch.Size([1, 24, 65])
torch.Size([1, 25, 65])
torch.Size([1, 26, 65])
torch.Size([1, 27, 65])
torch.Size([1, 28, 65])
torch.Size([1, 29, 65])
torch.Size([1, 30, 65])
torch.Size([1, 31, 65])
torch.Size([1, 32, 65])
torch.Size([1, 33, 65])
torch.Size([1, 34, 65])
torch.Size([1, 35, 65])
torch.Size([1, 36, 65])
torch.Size([1, 37, 65])
torch.Size([1, 38, 65])
torch.Size([1, 39, 65])
torch.Size([1, 40, 65])
torch.Size([1, 41, 65])
torch.Size([1, 42, 65])
t

In [89]:
a = torch.arange(18)
a.view(3*3, 2)[:, -1]
# print(a.view(4*8, 2))

tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17])